In [1]:
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import networkx as nx
import collections
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss
import warnings
from sklearn.svm import SVC
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from  sklearn.cluster import KMeans
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import  average_precision_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA


In [2]:
df = pd.read_csv("usersdata.csv", delimiter = '\t', names = ["ID", "Gender", "Time", "Age", "Label"], index_col = "ID")

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df

,Gender,Time,Age,Label
ID,,,,
1,M,0.9000,30,0
2,F,1.0000,20,0
3,M,0.1375,30,0
4,M,0.3875,20,0
5,M,0.0125,20,0
6,M,0.7125,20,0
7,M,0.9250,20,0
8,M,0.7875,20,0
9,M,0.0750,20,0


In [4]:
df["Label"].value_counts()

0    5270494
1     336953
Name: Label, dtype: int64

In [5]:
df.sample(frac = 0.01)["Label"].value_counts()

0    52626
1     3448
Name: Label, dtype: int64

In [6]:
# Assign weights to sample equal number of Spam and Ham
df["inv_value_count"] = 0
df["inv_value_count"][df.Label == 1] = 1/df["Label"].value_counts()[1]
df["inv_value_count"][df.Label == 0] = 1/df["Label"].value_counts()[0]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
# to sample equal number of spam/ham we add weight with inv_value_count columns
df_sample = df.sample(n = 50000, weights = 'inv_value_count').iloc[:, :4]

In [8]:
df_sample.head()

,Gender,Time,Age,Label
ID,,,,
4596575,M,0.0375,20,1
3336145,M,0.7000,20,0
2578000,F,0.0000,20,1
2793365,M,0.6250,20,1
1966669,M,0.7500,20,1


In [9]:
df_sample["Label"].value_counts()

0    25252
1    24748
Name: Label, dtype: int64

equal number of spam/ham users

In [10]:
# transform categorical features into one-hot-encoding
df_sample_preproc = pd.get_dummies(data = df_sample, columns = ["Age", "Gender"])

In [11]:
df_sample_preproc.head(5)

,Time,Label,Age_10,Age_20,Age_30,Age_40,Age_50,Age_60,Age_70,Gender_F,Gender_M
ID,,,,,,,,,,,
4596575,0.0375,1,0,1,0,0,0,0,0,0,1
3336145,0.7000,0,0,1,0,0,0,0,0,0,1
2578000,0.0000,1,0,1,0,0,0,0,0,1,0
2793365,0.6250,1,0,1,0,0,0,0,0,0,1
1966669,0.7500,1,0,1,0,0,0,0,0,0,1


In [12]:
Y_values = df_sample_preproc["Label"]
X_values = df_sample_preproc.drop(["Label"], axis = 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_values, Y_values, test_size=0.3)

## KNN

In [22]:
grid_params_knn = {
    'n_neighbors' : [3, 5, 11, 19, 29, 101, 501, 1001],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [23]:
gs_knn = GridSearchCV(KNeighborsClassifier(), grid_params_knn, verbose = 1, cv = 3, n_jobs = -1)
#gs_results_knn = gs_knn.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  4.3min finished


In [24]:
print(gs_results_knn.best_score_)
print(gs_results_knn.best_estimator_)
print(gs_results_knn.best_params_)


0.7204857142857143
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=101, p=2,
           weights='uniform')
{'metric': 'euclidean', 'n_neighbors': 101, 'weights': 'uniform'}


## Random Forest

In [27]:
grid_params_rf = {
    'bootstrap': [True],
    'max_depth': [2, 5, 10],
    'n_estimators': [10, 20, 50]
}
gs_rf = GridSearchCV(RandomForestClassifier(), grid_params_rf, verbose = 1, cv = 3, n_jobs = -1)
gs_results_rf = gs_rf.fit(X_train, y_train)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.8min


KeyboardInterrupt: 

In [ ]:
print(gs_results_rf.best_score_)
print(gs_results_rf.best_estimator_)
print(gs_results_rf.best_params_)


## Gradient Boosted Decision Tree

In [26]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [ ]:
grid_params_xgb = {
    'max_depth': [2, 5, 10],
    'n_estimators': [10, 20, 50, 100]
    'learning_rate': [0.1, 0.01, 0.05]
}

In [ ]:
gs_xgb = GridSearchCV(xgb, grid_params_xgb, verbose = 1, cv = 3, n_jobs = -1)
gs_results_xgb = gs_xgb.fit(X_train, y_train)